# Set Cover problem

In [ ]:
from random import random, seed
from itertools import product
import numpy as np
from tqdm.auto import tqdm

from icecream import ic

UNIVERSE_SIZE = 100_000
NUM_SETS = 10_000
DENSITY = 0.3

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

: 

## My solution

In [ ]:
def greedy_solution():
    """Genera una soluzione iniziale con un approccio greedy"""
    uncovered = np.ones(UNIVERSE_SIZE, dtype=bool)
    solution = np.zeros(NUM_SETS, dtype=bool)
    while np.any(uncovered):
        best_set = -1
        best_cost_benefit = float('inf')
        
        # Trova il miglior set che copre il maggior numero di elementi scoperti
        for i in range(NUM_SETS):
            if not solution[i]:  # Considera solo set non inclusi
                coverage = np.logical_and(SETS[i], uncovered).sum()
                if coverage > 0:
                    cost_benefit = COSTS[i] / coverage
                    if cost_benefit < best_cost_benefit:
                        best_set = i
                        best_cost_benefit = cost_benefit
        
        # Aggiungi il miglior set trovato alla soluzione
        solution[best_set] = True
        uncovered = np.logical_and(uncovered, np.logical_not(SETS[best_set]))
    
    return solution

# Miglioramento tramite tweak
def hill_climb(initial_solution):
    for n in tqdm(range(5_000)):  # Ridurre il numero di valutazioni
        temp = tweak(initial_solution)
        if valid(temp) and cost(temp) < cost(initial_solution):
            initial_solution = temp
    return initial_solution

# Soluzione iniziale con metodo greedy
initial_solution = greedy_solution()
ic(valid(initial_solution), cost(initial_solution))
